In [3]:
import warnings

# Desactivar todos los warnings
warnings.filterwarnings("ignore")

In [4]:
import os
import csv
import random

import mne
import time
import utils as u
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from scipy.signal import find_peaks

from pycrostates.cluster import ModKMeans
from pycrostates.preprocessing import extract_gfp_peaks
from pycrostates.io import ChData

import pickle

### Paths

In [3]:
# Define the path to the main data folder where the raw eeg can be found
DATA_FOLDER = 'E:\DATA_FOLDER'

# Define the path to the main folder where the microstate data will be stored
MICROSTATES = 'E:\MICROSTATES'

# Define the path to the main folder where the preprocessed data will be stored
DATA_PREPROCESS = 'E:\DATA_PREPROCESSED_MICROSTATES'
# Crear la carpeta para el paciente en la carpeta de destino
os.makedirs(DATA_PREPROCESS, exist_ok=True)

### Microstates

In [4]:
# Open the pickle file
with open(r"E:\M4_2500.pickle", 'rb') as file:
    ModK = pickle.load(file)

In [6]:
# Recording time required for the preprocessing
t = time.time()

# Loop through subject folders
for subject in os.listdir(DATA_FOLDER):
    print('\n', subject)

    # Check if the data has already been preprocessed
    if os.path.exists(os.path.join(MICROSTATES, subject)):
        #print(f"The files for the {subject} patient already exist. Going for the next patient.\n")
        continue

    else:
        # Construct the directory for each patient microstate information
        subject_dir = os.path.join(MICROSTATES, subject)
        
        # Construct the full path to the subject folder
        subject_path = os.path.join(DATA_FOLDER, subject)
    
        # Check if it's a directory
        if os.path.isdir(subject_path):
            print(f"\nProcessing data for subject {subject}...")
            
            # Construct the path to the BrainVision files folder
            rseeg_folder = os.path.join(subject_path, 'RSEEG')
            
            # Check if the RSEEG folder exists
            if os.path.exists(rseeg_folder):

                vhdr_files = [file for file in os.listdir(rseeg_folder) if file.endswith('.vhdr')]
                            
                # Check if a .vhdr file is found
                if vhdr_files:
                    
                    # Construct the full path to the .vhdr file
                    vhdr_path = os.path.join(rseeg_folder, vhdr_files[0])
                    
                    # Read the BrainVision files
                    try:
                        raw = mne.io.read_raw_brainvision(vhdr_path, eog=('VEOG',), preload=True, misc='auto', scale=1, verbose=None)
                        raw.set_montage("standard_1020")
                    except Exception as e:
                        print(f"Error reading files for subject {subject}: {e}")

                    # PREPROCESSING
                    # Cleaning EEG: Filtering and removing (ocular) artifacts
                    cleaned_eeg = u.preprocessing(raw)
                    data = cleaned_eeg._data
                    
                    # Dictionary with data
                    d = u.dictionary(cleaned_eeg)

                    # Samples corresponding to different events between "Closed Eyes" and "Open Eyes"
                    start_samples_ce, start_samples_oe, end_samples_ce, end_samples_oe = u.segment_samples(d)

                    
                    # Join the corresponding pairs and sort them in ascending order
                    pairs = u.pairs(start_samples_ce, start_samples_oe, end_samples_ce, end_samples_oe)

                    # Compute gfp, identify its peaks and valleys
                    _, peaks, valleys = u.gfp_data(cleaned_eeg)

                    # Compute microstate sequence and store it in a csv
                    u.microstate_segmentation(ModK, cleaned_eeg, peaks, valleys, pairs, subject_dir)
                    
                else:
                    print(f"No .vhdr file found for subject {subject}.")
            else:
                print(f"RSEEG folder not found for subject {subject}.")
        else:
            print(f"There is no subject {subject}.")

pt = time.time() - t
print('Tiempo de preprocesado', pt)


 sub-010002

Processing data for subject sub-010002...
Extracting parameters from E:\DATA_FOLDER\sub-010002\RSEEG\sub-010002.vhdr...
Setting channel info structure...
Reading 0 ... 2554999  =      0.000 ...  1022.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Using EOG channel: VEOG
EOG channel index for this subject is: [16]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel VEOG for blink detection
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 20.25 Hz)
- Filter length: 25000 sample

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Found 62 significant peaks
Number of EOG events detected: 62
Not setting metadata
62 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 62 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 88.7s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 2 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
There are 27686 peaks
There are 27685 valleys
CSV file created successfully.

 sub-010003

Processing data for subject sub-010003...
Extracting parameters from E:\DATA_FOLDER\sub-010003\RSEEG\sub-010003.vhdr...
Setting cha

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Number of EOG events detected: 374
Not setting metadata
374 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 374 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 35.0s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
There are 20765 peaks
There are 20764 valleys
CSV file created successfully.

 sub-010004

Processing data for subject sub-010004...
Extracting parameters from E:\DATA_FOLDER\sub-010004\RSEEG\sub-010004.vhdr...
Setting channel info structure...
R

...

Found 35 significant peaks
Number of EOG events detected: 35
Not setting metadata
35 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 35 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 24.0s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 1 ICA component
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S200', 'Stimulus/S210']
There are 24587 peaks
There are 24586 valleys
CSV file created successfully.

 sub-010319

Processing data for subject sub-010319...
Extracting parameters from E:\DATA_FOLDER\sub-010319\RSEEG\sub-010319.vhdr...
Setting channel info structur

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Number of EOG events detected: 34
Not setting metadata
34 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 34 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 34.3s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
There are 21432 peaks
There are 21431 valleys
CSV file created successfully.

 sub-010321

Processing data for subject sub-010321...
Extracting parameters from E:\DATA_FOLDER\sub-010321\RSEEG\sub-010321.vhdr...
Setting channel info structure...
Read

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Not setting metadata
65 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 65 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 21.5s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 4 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
There are 23521 peaks
There are 23520 valleys
CSV file created successfully.
Tiempo de preprocesado 13196.024129867554


### Preprocessing

In [5]:
# Define the microstate transitions
microstate_transitions = [
    "A-B", "A-C", "A-D", "B-A", "B-C", "B-D",
    "C-A", "C-B", "C-D", "D-A", "D-B", "D-C"
]

# Recording time required for the preprocessing
t = time.time()

# Loop through subject folders
for subject in os.listdir(DATA_FOLDER):
    print(subject)

    # Construct the full path to the subject folder
    subject_path = os.path.join(DATA_FOLDER, subject)
    
    # Check if it's a directory
    if os.path.isdir(subject_path):
        # Construct the path to the BrainVision files folder
        rseeg_folder = os.path.join(subject_path, 'RSEEG')
        
        # Check if the RSEEG folder exists
        if os.path.exists(rseeg_folder):

            vhdr_files = [file for file in os.listdir(rseeg_folder) if file.endswith('.vhdr')]
                            
            # Check if a .vhdr file is found
            if vhdr_files:
                
                # Construct the full path to the .vhdr file
                vhdr_path = os.path.join(rseeg_folder, vhdr_files[0])
                
                # Read the BrainVision files
                try:
                    raw = mne.io.read_raw_brainvision(vhdr_path, eog=('VEOG',), preload=True, misc='auto', scale=1, verbose=None)
                    raw.set_montage("standard_1020")
                except Exception as e:
                    print(f"Error reading files for subject {subject}: {e}")

                # PREPROCESSING
                # Cleaning EEG: Filtering and removing (ocular) artifacts
                cleaned_eeg = u.preprocessing(raw)
                data = cleaned_eeg._data
                
                # Dictionary with data
                d = u.dictionary(cleaned_eeg)

                # Samples corresponding to different events between "Closed Eyes" and "Open Eyes"
                start_samples_ce, start_samples_oe, end_samples_ce, end_samples_oe = u.segment_samples(d)

                # Segment division between "Closed Eyes" and "Open Eyes"
                segments_ce = u.division_segments(data, start_samples_ce, end_samples_ce)
                segments_oe = u.division_segments(data, start_samples_oe, end_samples_oe)

                # Segment store in a common list and transformation into a DataFrame
                segments = u.segments_df(cleaned_eeg, segments_ce, segments_oe)

                # Load microstate information
                microstate_file = os.path.join(MICROSTATES, f"{subject}_microstate.csv")
                if not os.path.exists(microstate_file):
                    print(f"No microstate file found for subject {subject}.")
                else:
                    microstates_df = pd.read_csv(microstate_file, delimiter=';')

                    # Create directories for each segment
                    for event_id in range(1, 17):
                        event_folder = os.path.join(DATA_PREPROCESS, f"segment_{event_id}")
                        os.makedirs(event_folder, exist_ok=True)

                        # Filter microstate information for the current event
                        event_microstates = microstates_df[microstates_df["event"] == event_id]

                        # Initialize a dictionary to hold dataframes for each transition
                        transition_dfs = {transition: pd.DataFrame() for transition in microstate_transitions}

                        for idx in range(len(event_microstates) - 1):
                            current_state = event_microstates.iloc[idx]["microstate"]
                            next_state = event_microstates.iloc[idx + 1]["microstate"]
                            transition = f"{current_state}-{next_state}"

                            if transition in microstate_transitions:
                                index = event_microstates.iloc[idx + 1]["index"]
                                start_idx = max(0, index - 13)
                                end_idx = min(data.shape[1], index + 13)
                                segment_data = data[:, start_idx:end_idx]
                                segment_df = pd.DataFrame(segment_data.T)

                                # Format the segment dataframe
                                formatted_segment_df = u.format_dataframe(segment_df)
                                
                                # Append the segment data to the corresponding transition dataframe
                                transition_dfs[transition] = pd.concat([transition_dfs[transition], segment_df], ignore_index=True)

                        # Save each transition dataframe as a CSV file
                        for transition, df in transition_dfs.items():
                            file_name = f"{subject}_{transition}.csv"
                            file_path = os.path.join(event_folder, file_name)
                            df.to_csv(file_path, index=False, sep='\t', header=False)

pt = time.time() - t
print('Preprocessing time:', pt)

sub-010002
Extracting parameters from E:\DATA_FOLDER\sub-010002\RSEEG\sub-010002.vhdr...
Setting channel info structure...
Reading 0 ... 2554999  =      0.000 ...  1022.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Using EOG channel: VEOG
EOG channel index for this subject is: [16]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel VEOG for blink detection
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 20.25 Hz)
- Filter length: 25000 samples (10.000 s)

Now detecting blinks and genera

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Found 62 significant peaks
Number of EOG events detected: 62
Not setting metadata
62 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 62 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 34.2s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 2 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
sub-010003
Extracting parameters from E:\DATA_FOLDER\sub-010003\RSEEG\sub-010003.vhdr...
Setting channel info structure...
Reading 0 ... 2519849  =      0.000 ...  1007.940 secs...
EEG channel type selected for re-referenc

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Number of EOG events detected: 374
Not setting metadata
374 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 374 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 28.9s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
sub-010004
Extracting parameters from E:\DATA_FOLDER\sub-010004\RSEEG\sub-010004.vhdr...
Setting channel info structure...
Reading 0 ... 2555099  =      0.000 ...  1022.040 secs...
EEG channel type selected for re-referencing
Applying average ref

...

Found 35 significant peaks
Number of EOG events detected: 35
Not setting metadata
35 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 35 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 37.7s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 1 ICA component
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S200', 'Stimulus/S210']
sub-010319
Extracting parameters from E:\DATA_FOLDER\sub-010319\RSEEG\sub-010319.vhdr...
Setting channel info structure...
Reading 0 ... 2537249  =      0.000 ...  1014.900 secs...
EEG channel type selected for re-referencing
Applying avera

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Not setting metadata
34 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 34 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 43.1s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
sub-010321
Extracting parameters from E:\DATA_FOLDER\sub-010321\RSEEG\sub-010321.vhdr...
Setting channel info structure...
Reading 0 ... 2597649  =      0.000 ...  1039.060 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) re

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Number of EOG events detected: 65
Not setting metadata
65 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 65 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 27.4s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 4 ICA components
    Projecting back using 61 PCA components
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']
Preprocessing time: 31487.651910305023


### Obtaining required file

In [6]:
source_path = r'H:\CSIC\DATA_PREPROCESSED_MICROSTATES'
target_path = r'H:\CSIC\DATA_PREPROCESSED_ROIS'
segment_number = 1
u.process_microstates_data(source_path, target_path, segment_number)

Data saved to H:\CSIC\DATA_PREPROCESSED_ROIS\segment_1.csv
